In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from baa import PerplexityBenchmark, get_llm_memory_usage, device_map
from datasets import load_dataset

In [ ]:
model_name = "HuggingFaceTB/SmolLM-360M-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device_map)
quantized_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)
franken_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print(model)

In [ ]:
counter = 0
for param in model.parameters():
    counter += 1
    print(param)
print(counter)